In [1]:
import pandas as pd
import sqlite3

In [ ]:
# Read the final cleaned CSVs
thermostat_hourly_df_sql = pd.read_csv("../data/processed/thermostat_hourly_custom.csv")
weather_df_sql = pd.read_csv("../data/processed/weather_data_cleaned_final.csv")

In [4]:

# Define SQLite database path (automatically create if doesn't exist)
db_path = "../database/thermostat_analysis.db"

# Connect to SQLite database
conn = sqlite3.connect(db_path)

# Create cursor to execute SQL commands
cursor = conn.cursor()

### Create Weather table with Primary Key

In [ ]:
# Ensure timestamp is datetime
weather_df_sql['timestamp'] = pd.to_datetime(weather_df_sql['timestamp'])

In [ ]:
# Convert timestamp to string (required for TEXT PRIMARY KEY)
weather_df_sql['timestamp'] = weather_df_sql['timestamp'].astype(str)


In [ ]:
# Create weather_data table

cursor.execute('''
CREATE TABLE IF NOT EXISTS weather_data_final (
    timestamp TEXT PRIMARY KEY,
    outdoor_humidity INTEGER,
    wind_speed_kmh REAL,
    outdoor_temp_f REAL
)
''')

In [ ]:
# Insert DataFrame into the table
weather_df_sql.to_sql('weather_data_final', conn, if_exists='replace', index=False)

10224

### Create Thermostat hourly data table with Primary Key

In [5]:
# Ensure timestamp is datetime
thermostat_hourly_df_sql['timestamp'] = pd.to_datetime(thermostat_hourly_df_sql['timestamp'])

In [6]:
# Convert timestamp to string (required for TEXT PRIMARY KEY)
thermostat_hourly_df_sql['timestamp'] = thermostat_hourly_df_sql['timestamp'].astype(str)

In [7]:
# Create thermostat_data_hourly table
cursor.execute('''
CREATE TABLE IF NOT EXISTS thermostat_data_hourly (
    timestamp TEXT PRIMARY KEY,
    system_mode TEXT,
    program_mode TEXT,
    cool_set_temp_f REAL,
    heat_set_temp_f REAL,
    current_temp_f REAL,
    current_humidity_rh REAL,
    fan_sec REAL,
    fan_runtime_category TEXT
)
''')

In [8]:
# Insert DataFrame into the table
thermostat_hourly_df_sql.to_sql('thermostat_data_hourly', conn, if_exists='replace', index=False)

10224

In [9]:
# Check if tables exist in the db

tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print(tables)

                     name
0            weather_data
1      weather_data_final
2   thermostat_data_final
3  thermostat_data_hourly


In [ ]:
# Confirm Data was Inserted
# Quick query to check contents in the table
result = pd.read_sql("SELECT * FROM weather_data_final LIMIT 5;", conn)
print(result)

             timestamp  outdoor_humidity  wind_speed_kmh  outdoor_temp_f
0  2024-05-01 00:00:00                75             6.1           58.46
1  2024-05-01 01:00:00                80             5.5           56.30
2  2024-05-01 02:00:00                83             4.8           55.04
3  2024-05-01 03:00:00                86             7.4           54.14
4  2024-05-01 04:00:00                88             9.3           54.14


In [11]:
# Confirm Data was Inserted
# Quick query to check contents in the table
result = pd.read_sql("SELECT * FROM thermostat_data_hourly LIMIT 5;", conn)
print(result)

             timestamp  current_temp_f  cool_set_temp_f  heat_set_temp_f  \
0  2024-05-01 00:00:00           72.88             74.5             69.5   
1  2024-05-01 01:00:00           72.43             74.5             69.5   
2  2024-05-01 02:00:00           72.05             74.5             69.5   
3  2024-05-01 03:00:00           71.59             74.5             69.5   
4  2024-05-01 04:00:00           71.17             74.5             69.5   

   current_humidity_rh  fan_sec        system_mode program_mode  \
0                 57.0   3300.0  compressorcooloff        sleep   
1                 57.0   3300.0  compressorcooloff        sleep   
2                 57.0   3300.0  compressorcooloff        sleep   
3                 57.0   3300.0  compressorcooloff        sleep   
4                 57.0   3300.0  compressorcooloff        sleep   

  fan_runtime_category  
0                 High  
1                 High  
2                 High  
3                 High  
4              

In [ ]:
# Run PRAGMA to get schema details
schema_df = pd.read_sql("PRAGMA table_info(weather_data_final);", conn)

#Show schema
print(schema_df)

   cid              name     type  notnull dflt_value  pk
0    0         timestamp     TEXT        0       None   0
1    1  outdoor_humidity  INTEGER        0       None   0
2    2    wind_speed_kmh     REAL        0       None   0
3    3    outdoor_temp_f     REAL        0       None   0


In [12]:
# Run PRAGMA to get schema details
schema_df = pd.read_sql("PRAGMA table_info(thermostat_data_hourly);", conn)

#Show schema
print(schema_df)

   cid                  name  type  notnull dflt_value  pk
0    0             timestamp  TEXT        0       None   0
1    1        current_temp_f  REAL        0       None   0
2    2       cool_set_temp_f  REAL        0       None   0
3    3       heat_set_temp_f  REAL        0       None   0
4    4   current_humidity_rh  REAL        0       None   0
5    5               fan_sec  REAL        0       None   0
6    6           system_mode  TEXT        0       None   0
7    7          program_mode  TEXT        0       None   0
8    8  fan_runtime_category  TEXT        0       None   0


In [13]:
# Commit the transaction and close the connection

conn.commit()
conn.close()